In [39]:
from os import listdir
from os.path import isfile, join
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import pickle
from gensim.similarities import MatrixSimilarity
from gensim.summarization.summarizer import summarize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ronnald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
with open('label_content', 'rb') as f:
    labelContent = pickle.load(f)

In [41]:
tagged_data = [TaggedDocument(words=word_tokenize(content.lower()), tags=[str(label)]) for label, content in labelContent.items()]

In [5]:
#tagged_data

In [42]:
max_epochs = 10
vec_size = 100#neurons
alpha = 0.025
window = 3

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                window = window)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    #model.min_alpha = model.alpha

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [26]:
def parseMatrix(doc1,model):
    for j in range(0,len(tagged_data)):
        doc2 = tagged_data[j][1][0]
        distance[i][j]=model.docvecs.similarity(doc1,doc2)

In [10]:

def parseRelations(doc,most_similar):
    #Get top 5
    docRelation[doc] = {}
    for i in range(0,5):
        doc2,rel = most_similar[i]
        docRelation[doc][doc2]={'weight':rel}

In [33]:

from time import time
import numpy as np
start = time()


distance =np.zeros(shape=(len(tagged_data),len(tagged_data)))
for i in range(0,len(tagged_data)):
    parseMatrix(tagged_data[i][1][0],model)
        
print('Took %.2f seconds to run.' % (time() - start))

/home/ronnald/dev/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Took 15.35 seconds to run.


In [44]:
docRelation ={}
for i in range(0,len(tagged_data)):
    parseMatrix(tagged_data[i][1][0],model)
    if(model.docvecs.most_similar(i)[0][1] > 0.65):
        parseRelations(str(tagged_data[i][1][0]),model.docvecs.most_similar(i))
        
with open('graph', 'wb') as f:
    pickle.dump(docRelation, f)

/home/ronnald/dev/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [17]:

#https://stackoverflow.com/questions/39455741/gcc-error-trying-to-exec-cc1plus-execvp-no-such-file-or-directory

In [10]:
with open('label_sentence', 'rb') as f:
    labelSentence = pickle.load(f)

In [21]:
print(summarize(labelSentence["2017-2-261451"],ratio=0.2))

Essa imposicao das crencas pessoais sobre o outro, como se elas fossem uma verdade absoluta, que nao pode ser contestada, e o fundamentalismo.
Essa repressao e esse preconceito contra diferentes religioes e seus cultos faz com que muitos nao se sintam a vontade para expressar sua fe e se declararem parte de algum grupo religioso.
Muitas pessoas seguem a religiao que sua familia faz parte, so por tradicao, o que e muito comum nas familias catolicas.
Na minha opiniao, nao e um problema que a religiao seja heranca familiar, mas e um problema que ela seja imposta, sem que a pessoa possa procurar e conhecer outras religioes e entao decidir qual delas e mais adequada a seus valores.


In [22]:
print(summarize(labelSentence["2017-1-181811"],ratio=0.2))

Ha tambem pessoas que aderiram a outras religioes por diversos motivos como: localizacao geografica e afinidade com a doutrina, pois muitas vezes existe a influencia familiar que determina a religiao.
Ao mesmo tempo que a diversidade religiosa propicia liberdade para que a pessoa possa escolher o caminho a ser seguido, isso traz intolerancia por parte de algumas pessoas que julgam que a sua religiao e melhor do que a do outro.
Porem, quanto a intolerancia, as pessoas que nao tem uma opiniao formada sobre  determinada religiao acabam acatando a falta de respeito dos grupos intolerantes.


In [36]:
with open('cosine_distance', 'wb') as f:
    pickle.dump(distance, f)